In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install tensorflow

In [3]:
# !pip install sklearn

In [4]:
# !pip install missingno

# 1. 데이터 로드 및 확인

In [5]:
# import libraries
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import os
import numpy as np
import pandas as pd
import math, random
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# import seaborn as sns
# sns.set_style('whitegrid')
from scipy import stats
# import missingno as msno

# display all columns
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_row', None)

In [6]:
# load data
path = '/Users/jsha/gjai/final_project/black_swan_physionet/'
train_data = pd.read_csv(path + 'physionet2012_train.csv')
valid_data = pd.read_csv(path + 'physionet2012_valid.csv')
test_data = pd.read_csv(path + 'physionet2012_test.csv')

train_data.drop(['Unnamed: 0'], axis=1, inplace=True)
valid_data.drop(['Unnamed: 0'], axis=1, inplace=True)
test_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242613 entries, 0 to 242612
Data columns (total 45 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   pid          242613 non-null  int64  
 1   rec_time     242613 non-null  object 
 2   age          242613 non-null  float64
 3   Gender       242613 non-null  float64
 4   ICUType      242613 non-null  float64
 5   ALP          2480 non-null    float64
 6   ALT          2549 non-null    float64
 7   AST          2552 non-null    float64
 8   Albumin      1892 non-null    float64
 9   BUN          11220 non-null   float64
 10  Bilirubin    2551 non-null    float64
 11  Cholesterol  273 non-null     float64
 12  Creatinine   11274 non-null   float64
 13  FiO2         26127 non-null   float64
 14  GCS          49959 non-null   float64
 15  Glucose      10513 non-null   float64
 16  HCO3         10972 non-null   float64
 17  HCT          14762 non-null   float64
 18  HR           242613 non-

In [8]:
# labeling
# train_data
train_data['label'] = 0
train_data.loc[(2 <= train_data['survival'])&(train_data['survival'] <= train_data['LOS']), 'label'] = 1
train_data.loc[train_data['survival'] > train_data['LOS'], 'label'] = 0
train_data.loc[train_data['survival'] == -1, 'label'] = 0

# valid_data
valid_data['label'] = 0
valid_data.loc[(2 <= valid_data['survival'])&(valid_data['survival'] <= valid_data['LOS']), 'label'] = 1
valid_data.loc[valid_data['survival'] > valid_data['LOS'], 'label'] = 0
valid_data.loc[valid_data['survival'] == -1, 'label'] = 0

# test_data
test_data['label'] = 0
test_data.loc[(2 <= test_data['survival'])&(test_data['survival'] <= test_data['LOS']), 'label'] = 1
test_data.loc[test_data['survival'] > test_data['LOS'], 'label'] = 0
test_data.loc[test_data['survival'] == -1, 'label'] = 0

In [9]:
train_df = train_data.copy()
valid_df = valid_data.copy()
test_df = test_data.copy()

# 2. 결측값 처리

> **df.fillna(value=None, method=None, axis=None, inplace=False, limit=None, downcast=None)**
* value : 결측값을 대체할 값입니다. dict형태로도 가능합니다.
* method : 결측값을 변경할 방식입니다. bfill로 할경우 결측값을 바로 아래 값과 동일하게 변경합니다.
ffill로 할 경우 결측값을 바로 위 값과 동일하게 변경합니다.
* axis : {0 : index / 1 : columns} fillna 메서드를 적용할 레이블입니다.
* inplace : 원본을 변경할지 여부입니다. True일 경우 원본을 변경하게 됩니다.
* limit : 결측값을 변경할 횟수입니다. 위에서부터 limit로 지정된 갯수만큼만 변경합니다.
* downcast : 다운캐스트할지 여부입니다. downcast='infer'일 경우 float64를 int64로 변경합니다.

In [10]:
# ffill
train_df.groupby(['pid']).fillna(method='ffill', inplace=True)
valid_df.groupby(['pid']).fillna(method='ffill', inplace=True)
test_df.groupby(['pid']).fillna(method='ffill', inplace=True)

""


In [11]:
# median
train_df.fillna(train_df.median(), inplace=True)
valid_df.fillna(valid_df.median(), inplace=True)
test_df.fillna(test_df.median(), inplace=True)

# 3. 데이터 확인

In [12]:
train_df.describe()

,pid,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,Tropl,TroponinI,TropT,TroponinT,Urine,WBC,pH,MechVent,survival,SBP,DBP,MBP,male,female,surv_los,LOS,label
count,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,242613.000000,0.0,242613.000000,0.0,242613.000000,242613.000000,242613.000000,242613.000000,242613.0,242613.000000,242613.000000,242613.000000,242613.000000,242613.0,242613.0,242613.000000,242613.000000,242613.000000
mean,137598.163227,64.660979,0.567748,2.721355,82.363727,46.252200,67.431234,2.900154,20.333844,0.920969,154.004942,1.023582,0.504850,12.674613,127.632637,23.004035,30.325126,87.772782,4.101816,2.121667,2.001087,139.001224,39.113103,124.225732,173.877780,19.154167,96.883451,36.977602,NaN,2.907237,NaN,0.227553,92.485370,11.549491,7.389980,1.0,127.016673,116.577113,58.631030,79.120566,1.0,0.0,112.964009,14.052664,0.140553
std,2895.568188,17.069362,0.497176,1.003291,14.178149,112.848326,159.476285,0.057804,5.217707,0.636930,1.526859,0.375091,0.063831,1.906861,14.596424,1.008024,1.249393,18.031186,0.155843,0.433795,0.086192,1.083102,2.547023,26.050993,23.496668,2.419724,2.256985,1.490571,NaN,0.443048,NaN,0.254353,124.691656,1.506942,2.593948,0.0,361.568613,22.507286,12.065476,15.948036,0.0,0.0,361.025804,12.713974,0.347561
min,132540.000000,15.000000,-1.000000,1.000000,12.000000,1.000000,5.000000,1.000000,0.000000,0.100000,28.000000,0.100000,0.210000,3.000000,10.000000,5.000000,10.300000,0.000000,1.800000,0.300000,0.600000,98.000000,0.300000,0.000000,6.000000,10.000000,26.000000,-17.800000,NaN,0.300000,NaN,0.010000,0.000000,0.100000,1.000000,1.0,-1.000000,0.000000,0.000000,0.000000,1.0,0.0,-105.000000,-1.000000,0.000000
25%,135127.000000,53.000000,0.000000,2.000000,82.000000,43.000000,63.000000,2.900000,20.000000,0.900000,154.000000,1.000000,0.500000,13.000000,127.000000,23.000000,30.300000,76.000000,4.100000,2.100000,2.000000,139.000000,39.000000,122.000000,173.000000,19.000000,97.000000,36.500000,NaN,2.900000,NaN,0.220000,70.000000,11.500000,7.380000,1.0,-1.000000,108.000000,54.000000,73.000000,1.0,0.0,-13.000000,6.000000,0.000000
50%,137594.000000,67.000000,1.000000,3.000000,82.000000,43.000000,63.000000,2.900000,20.000000,0.900000,154.000000,1.000000,0.500000,13.000000,127.000000,23.000000,30.300000,86.000000,4.100000,2.100000,2.000000,139.000000,39.000000,122.000000,173.000000,19.000000,97.000000,37.100000,NaN,2.900000,NaN,0.220000,70.000000,11.500000,7.380000,1.0,-1.000000,116.000000,58.000000,77.000000,1.0,0.0,-7.000000,10.000000,0.000000
75%,140031.000000,78.000000,1.000000,4.000000,82.000000,43.000000,63.000000,2.900000,20.000000,0.900000,154.000000,1.000000,0.500000,13.000000,127.000000,23.000000,30.300000,98.000000,4.100000,2.100000,2.000000,139.000000,39.000000,122.000000,173.000000,19.000000,97.000000,37.600000,NaN,2.900000,NaN,0.220000,70.000000,11.500000,7.380000,1.0,15.000000,124.000000,62.000000,83.000000,1.0,0.0,-1.000000,17.000000,0.000000
max,142671.000000,90.000000,1.000000,4.000000,2205.000000,11470.000000,18430.000000,5.000000,197.000000,47.700000,330.000000,22.100000,1.000000,15.000000,1143.000000,50.000000,61.800000,300.000000,22.900000,29.300000,7.700000,177.000000,100.000000,500.000000,1047.000000,39.000000,100.000000,42.100000,NaN,49.200000,NaN,24.910000,11000.000000,187.500000,735.000000,1.0,2600.000000,295.000000,268.000000,300.000000,1.0,0.0,2547.000000,154.000000,1.000000


In [13]:
train_df['label'].value_counts()

0    208513
1     34100
Name: label, dtype: int64

In [14]:
train_df

,pid,rec_time,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,Tropl,TroponinI,TropT,TroponinT,Urine,WBC,pH,MechVent,survival,SBP,DBP,MBP,male,female,surv_los,LOS,label
0,142219,0:00:00,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,86.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,NaN,2.9,NaN,0.22,70.0,11.5,7.38,1.0,-1,116.0,58.0,77.0,1,0,-8,7,0
1,142219,0:44:00,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.8,13.0,127.0,23.0,30.3,103.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,NaN,2.9,NaN,0.22,500.0,11.5,7.38,1.0,-1,116.0,58.0,77.0,1,0,-8,7,0
2,142219,1:14:00,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,100.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,NaN,2.9,NaN,0.22,70.0,11.5,7.38,1.0,-1,0.0,0.0,224.0,1,0,-8,7,0
3,142219,1:44:00,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,106.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,NaN,2.9,NaN,0.22,70.0,11.5,7.38,1.0,-1,0.0,0.0,135.0,1,0,-8,7,0
4,142219,1:54:00,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,105.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,NaN,2.9,NaN,0.22,450.0,11.5,7.38,1.0,-1,217.0,108.0,133.0,1,0,-8,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,"1 day, 20:36:00",53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.4,8.0,127.0,23.0,30.3,85.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,NaN,2.9,NaN,0.22,80.0,11.5,7.38,1.0,17,114.0,65.0,84.0,1,0,-1,18,1
242609,138581,"1 day, 21:36:00",53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,83.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,NaN,2.9,NaN,0.22,40.0,11.5,7.38,1.0,17,111.0,61.0,80.0,1,0,-1,18,1
242610,138581,"1 day, 22:36:00",53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,83.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,NaN,2.9,NaN,0.22,45.0,11.5,7.38,1.0,17,109.0,62.0,80.0,1,0,-1,18,1
242611,138581,"1 day, 23:36:00",53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,85.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,NaN,2.9,NaN,0.22,50.0,11.5,7.38,1.0,17,111.0,63.0,80.0,1,0,-1,18,1


In [15]:
# 0:여성 1:남성
train_df['Gender'].unique()

array([ 0.,  1., -1.])

In [16]:
train_df[train_df['Gender']==-1]['pid'].unique()
# 남자...?
# 141486 -> male:1 female:0
# 137392 -> male:1 female:0
# 135757 -> male:1 female:0

array([141486, 137392, 135757])

In [17]:
# Gender -1값 1로 바꾸기
train_df.loc[train_df['Gender']==-1, 'Gender'] = 1 

In [18]:
# 1:관상동맥 2:심장 3:내과 4:외과
train_df['ICUType'].unique()

array([3., 4., 2., 1.])

In [19]:
train_df.drop(columns=['rec_time', 'Tropl', 'TropT', 'male', 'female', 'survival', 'surv_los', 'LOS'], inplace=True)
valid_df.drop(columns=['rec_time', 'Tropl', 'TropT', 'male', 'female', 'survival', 'surv_los', 'LOS'], inplace=True)
test_df.drop(columns=['rec_time', 'Tropl', 'TropT', 'male', 'female', 'survival', 'surv_los', 'LOS'], inplace=True)

# 4. 데이터 시각화 

In [20]:
train_df['label'].value_counts()/len(train_df.index)

0    0.859447
1    0.140553
Name: label, dtype: float64

In [21]:
train_df[train_df['label']==1]['ICUType'].value_counts()/(train_df[train_df['label']==1]['ICUType'].value_counts()+train_df[train_df['label']==0]['ICUType'].value_counts())

1.0    0.167024
2.0    0.050572
3.0    0.186448
4.0    0.159788
Name: ICUType, dtype: float64

In [22]:
# categorical data
def bar_chart(feature, ax=None):
    dead = train_df[train_df['label']==1][feature].value_counts() 
    survived = train_df[train_df['label']==0][feature].value_counts()
    df = pd.DataFrame([dead, survived])
    df.index = ['Dead', 'Survived']
    df.plot(kind='bar', stacked=True, ax=ax)

# numerical data(facets)
def draw_facetgrid(feature):
    facet = sns.FacetGrid(train_df, hue='label', aspect=5)
    facet.map(sns.kdeplot, feature, shade=True)
    facet.set(xlim=(0, train_df[feature].max()))
    facet.add_legend()
    plt.show()

In [23]:
def count_plot(column, ax):
    sns.countplot(x=column, hue='label', data=train_df, ax=ax)

# 5. 데이터 패딩

In [24]:
gr_train_df = train_df.groupby('pid')
gr_valid_df = valid_df.groupby('pid')
gr_test_df = test_df.groupby('pid')

In [25]:
gr_train_df.size()
gr_valid_df.size()
gr_test_df.size()

pid
132551    94
132567    75
132570    46
132582    66
132584    94
          ..
142561    88
142573    62
142576    90
142595    45
142641    48
Length: 400, dtype: int64

In [49]:
train_df[train_df['pid']==142219].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
           dtype='int64')

In [27]:
# train_df[train_df['pid']==ep]['age'][0]
# train_df[train_df['pid']==ep]['age'].values[0]

In [36]:
train_df

,pid,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,TroponinI,TroponinT,Urine,WBC,pH,MechVent,SBP,DBP,MBP,label
0,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,86.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,70.0,11.5,7.38,1.0,116.0,58.0,77.0,0
1,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.8,13.0,127.0,23.0,30.3,103.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,500.0,11.5,7.38,1.0,116.0,58.0,77.0,0
2,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,100.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,70.0,11.5,7.38,1.0,0.0,0.0,224.0,0
3,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,106.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,70.0,11.5,7.38,1.0,0.0,0.0,135.0,0
4,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,105.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,450.0,11.5,7.38,1.0,217.0,108.0,133.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.4,8.0,127.0,23.0,30.3,85.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,80.0,11.5,7.38,1.0,114.0,65.0,84.0,1
242609,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,83.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,40.0,11.5,7.38,1.0,111.0,61.0,80.0,1
242610,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,83.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,45.0,11.5,7.38,1.0,109.0,62.0,80.0,1
242611,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,85.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,50.0,11.5,7.38,1.0,111.0,63.0,80.0,1


In [29]:
test = pd.DataFrame([[2,1,1,1,1],
                    [2,2,2,2,2],
                    [2,3,3,3,3]], columns=list('abcde'))

# test[test['a']==2] = test[test['a']==2][:2] 
test
test[test['d']==3]

# test = pd.Series([0,0,0])
test['a'] = 100
test



,a,b,c,d,e
0,100,1,1,1,1
1,100,2,2,2,2
2,100,3,3,3,3


In [50]:
# train_df_original = train_df.copy() 
pad_train_df = pd.DataFrame(columns=train_df.columns)
p = train_df['pid'].unique()
max_len = 70
for ep in p:
    if len(train_df[train_df['pid']==ep]) >= max_len:
        new_df = train_df[train_df['pid']==ep][:max_len]
        pad_train_df = pd.concat([pad_train_df, new_df])

    else:
        empty_df = pd.DataFrame(index=range(0,70), columns = train_df.columns) 
        empty_df['pid'] = ep
        empty_df['age'] = train_df[train_df['pid']==ep]['age'].values[0]
        empty_df['Gender'] = train_df[train_df['pid']==ep]['Gender'].to_numpy()[0]
        empty_df['ICUType'] = train_df[train_df['pid']==ep]['ICUType'].values[0]
        empty_df['label'] = train_df[train_df['pid']==ep]['label'].to_list()[0]
        empty_df[-len(train_df[train_df['pid']==ep]):] = train_df[train_df['pid']==ep]
        empty_df.fillna(0, inplace=True)
        pad_train_df = pd.concat([pad_train_df, empty_df])

In [114]:
pd.set_option('display.max_rows', None)
pd.DataFrame(train_df.groupby(by='pid').size())

,0
pid,
132540.0,70
132555.0,70
132556.0,70
132575.0,70
132585.0,70
132599.0,70
132601.0,70
132612.0,70
132615.0,70


# 6. 범주형 데이터 원핫인코딩

In [109]:
train_df = pd.get_dummies(train_df, columns=['Gender', 'ICUType'])
valid_df = pd.get_dummies(valid_df, columns=['Gender', 'ICUType'])
test_df = pd.get_dummies(test_df, columns=['Gender', 'ICUType'])

In [110]:
train_df
# pid, Gender_0.0, Gender_1.0, label

,pid,age,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,TroponinI,TroponinT,Urine,WBC,pH,MechVent,SBP,DBP,MBP,label,Gender_0.0,Gender_1.0,ICUType_1.0,ICUType_2.0,ICUType_3.0,ICUType_4.0
0,142219,52.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,86.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,70.0,11.5,7.38,1.0,116.0,58.0,77.0,0,1,0,0,0,1,0
1,142219,52.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.8,13.0,127.0,23.0,30.3,103.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,500.0,11.5,7.38,1.0,116.0,58.0,77.0,0,1,0,0,0,1,0
2,142219,52.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,100.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,70.0,11.5,7.38,1.0,0.0,0.0,224.0,0,1,0,0,0,1,0
3,142219,52.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,106.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,70.0,11.5,7.38,1.0,0.0,0.0,135.0,0,1,0,0,0,1,0
4,142219,52.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,105.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,97.0,37.1,2.9,0.22,450.0,11.5,7.38,1.0,217.0,108.0,133.0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,53.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.4,8.0,127.0,23.0,30.3,85.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,80.0,11.5,7.38,1.0,114.0,65.0,84.0,1,0,1,0,0,0,1
242609,138581,53.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,83.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,40.0,11.5,7.38,1.0,111.0,61.0,80.0,1,0,1,0,0,0,1
242610,138581,53.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,83.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,45.0,11.5,7.38,1.0,109.0,62.0,80.0,1,0,1,0,0,0,1
242611,138581,53.0,82.0,43.0,63.0,2.9,20.0,0.9,154.0,1.0,0.5,13.0,127.0,23.0,30.3,85.0,4.1,2.1,2.0,139.0,39.0,122.0,173.0,19.0,100.0,36.9,2.9,0.22,50.0,11.5,7.38,1.0,111.0,63.0,80.0,1,0,1,0,0,0,1


In [111]:
ordinary_train = train_df.copy()
ordinary_valid = valid_df.copy()
ordinary_test = test_df.copy()

# 7. 데이터 정규화

In [112]:
ordinary_train.drop(columns=['pid', 'label'], inplace=True)
ordinary_valid.drop(columns=['pid', 'label'], inplace=True)
ordinary_test.drop(columns=['pid', 'label'], inplace=True)

In [113]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(ordinary_train)
scaled_train = scaler.transform(ordinary_train)
scaled_valid = scaler.transform(ordinary_valid)
scaled_test = scaler.transform(ordinary_test)
# scaled_train_df = pd.DataFrame(data=scaled_train, columns=ordinary_train.feature_names)